# Ensemble-FF-Fit - Ensemble Fitting of MACE

In this tutorial, we will demonstrate how we can use the [Ensemble-FF-Fit](https://github.com/Q-CAD/Ensemble-FF-Fit) and [MatEnsemble](https://github.com/Q-CAD/MatEnsemble) packages to fine-tune a [MACE](https://github.com/ACEsuit/mace) force field foundation model [MACE-OMAT](https://github.com/ACEsuit/mace-foundations). This example will fit force fields for bulk phases in the Pd-Se compositional system, and is meant to be run on the [Perlmutter](https://docs.nersc.gov/systems/perlmutter/architecture) supercomputer. 

This notebook was written by Ryan Morelock.

## Outline of this notebook:

### Installation
1. **Build a Conda environment** supporting Ensemble-FF-Fit
2. **Create a Jupyter kernel** for this environment
3. **Write a helper script** to load necessary dependencies, and update the ```kernel.json``` file
4. **Check that the necessary executables** can be loaded

### A. Fitting an Ensemble of Force Fields on HPC
1. **Generate Pd-Se structures** to be used as training and testing data
2. **Perform DFT calculations** on these modified structures using [vaspFlux](https://code.ornl.gov/rym/vaspflux)
3. **Parse MACE compatible input files** from DFT outputs using [parse2fit](https://code.ornl.gov/rym/parse2fit)
4. **Fit an ensemble of MACE force fields** using [MACE-freeze](https://github.com/7radians/mace-freeze)

### B. Uncertainty Quantification for Additional Fitting
1. **Run Pd-Se melting simulations** on structures in training data
2. **Determine the images with the greatest uncertainty** along these trajectories using the force field ensemble
3. **Calculate DFT single-point energies** for the most uncertain, to be added as new training data
4. **Determine the best force field** from the ensemble based on DFT validation

# Installation

## 1. Build a Conda environment

Ensemble-FF-Fit and its dependencies can be installed into a conda environment using the build scripts provided with the distribution. 

```git clone https://github.com/Q-CAD/Ensemble-FF-Fit/tree/main```

The conda environment can then be built by navigating into the ```Ensemble-FF-Fit``` directory and running

```. {chosen_build}.sh /desired/path/to/conda/environment```

## 2. Create a Jupyter kernel

A Jupyter kernel used to launch this notebook can be created after activating the built environment, for example:

```conda activate {your name}```

or your conda environment alias. Next, make the environment loadable as a Jupyter kernel:

```python -m ipykernel install --user --name {your name} --display-name "Your Name"```

## 3. Write a helper script

[MatEnsemble](https://github.com/Q-CAD/MatEnsemble) uses [Flux](https://flux-framework.readthedocs.io/en/latest/guides/learning_guide.html) to dynamically coordinate jobs on HPCs based on the resources available at runtime. 

Flux with MatEnsemble supported has currently been installed with [Spack](https://spack.io) on Perlmutter at:

```/global/cfs/cdirs/m5014/spack_py3.11/spack/share/spack/setup-env.sh```

We will need to load this environment, which contains its own Python, so that it can run alongside the Python installed into our Jupyter kernel. 

This can be accomplished via [helper-script](https://docs.nersc.gov/services/jupyter/how-to-guides/) formatting, as discussed in the Perlmutter documentation. An example helper script, which loads Cuda toolkit to support calculations on GPUs, as well as VASP module on Perlmutter, is provided below. 

In [1]:
helper_script = '''
#!/bin/bash
set -e

# Ensure module command exists
# source /etc/profile
# source /usr/share/Modules/init/bash

# Load Cray + CUDA stack
module load PrgEnv-gnu/8.5.0
module load cudatoolkit/12.4
module load craype-accel-nvidia80

# (Optional) Only load python module if you need it for tools
# Do NOT rely on it for the kernel Python
module load python/3.11
module load vasp/6.4.3-gpu

# Activate the spack
source /global/cfs/cdirs/m5014/spack_py3.11/spack/share/spack/setup-env.sh
spack load flux-sched
unset LUA_PATH LUA_CPATH

# Set the variables
CONDA_ENV="/global/homes/r/rym/.conda/envs/unified"
CONDA_SITE="$CONDA_ENV/lib/python3.11/site-packages"
CONDA_PYTHON="$CONDA_ENV/bin/python"

# Load the conda environment
conda activate $CONDA_ENV

# Add conda environment to PYTHONPATH before the Spack path
export PYTHONPATH="${CONDA_SITE}:${PYTHONPATH}"

exec "$@"
'''

This script must be made executable, using a command like: ```chmod u+x $HOME/.local/share/jupyter/kernels/env/kernel-helper.sh```

Finally, the ```kernel.json``` file will need to be updated so that the helper script is loaded. The script below forces the Python module that the Spack was compiled against to be the default, which is the current "hacky" way to get the MatEnsemble + Flux functionality.   

In [2]:
example_json = '''{
 "argv": [
  "{resource_dir}/kernel-helper.sh",
  "python", # Forces the loaded Python module to be used
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "Unified",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}'''

## 4. Confirm the necessary executables

You should now be able to restart the notebook and load the kernel without issues (check the unfilled circle in upper right hand corner). 

We can also check that Flux is accessible based on our helper script that has been added to the location with the Jupyter Kernel. 

In [3]:
import sys, os, subprocess

print(sys.executable)
print(sys.version)

!which flux
!flux version

/global/cfs/cdirs/m5014/spack_py3.11/spack/opt/spack/linux-zen3/python-venv-1.0-52phvnnncpahv4yrmtaoif6xqifqfyez/bin/python
3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0]
/global/cfs/cdirs/m5014/spack_py3.11/spack/opt/spack/linux-zen3/flux-core-0.73.0-5mhgz76sun6xxy6ybayeu55dut7gq325/bin/flux
commands:    		0.73.0
libflux-core:		0.73.0
build-options:		+systemd+hwloc.api==2.11.0+zmq==4.3.5


Finally, let's prepend the Ensemble-FF-Fit bin location to the ```PATH``` variable so we can use the Python executables provided; using these executables will make certain parts of the workflow much easier. 

In [4]:
import sys, os, shutil, subprocess, glob

In [5]:
def remove_path(path):
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
            return 
        os.remove(path)
    return 

Let's check that the ```mp_query``` executable is available to load, and if it is, all executables should be loadable.

In [6]:
env_bin = os.path.dirname(sys.executable)
os.environ["PATH"] = env_bin + os.pathsep + os.environ.get("PATH", "")

print("mp_query executable location:", shutil.which("mp_query"))

mp_query executable location: /global/homes/r/rym/.conda/envs/unified/bin/mp_query


# A. Fitting an Ensemble of Force Fields on HPC

## 1, 2. Generate Structures and Run DFT Calculations

We'll start by defining a couple of useful functions for file reading, writing and checking.

In [7]:
import yaml
import shutil
import json 

def write_json(path, dct):
    with open(path, 'w') as outfile:
        json.dump(dct, outfile, indent=4)

def write_yaml(path, dct):
    with open(path, 'w') as outfile:
        yaml.dump(dct, outfile, default_flow_style=False, sort_keys=False)
    return 

def write_string(path, string):
    with open(path, 'w') as outfile:
        outfile.write(string)

def read_file(path):
    with open(path, 'r') as infile:
        lst = infile.readlines()
    return lst

def copy_files(src_dir, dst_dir, patterns=()):
    os.makedirs(dst_dir, exist_ok=True)

    for fname in os.listdir(src_dir):
        if any(pat in fname for pat in patterns):
            src_path = os.path.join(src_dir, fname)
            dst_path = os.path.join(dst_dir, fname)

            if os.path.isfile(src_path):
                shutil.copy(src_path, dst_path)

def file_exists_anywhere(root_dir, filename):
    for _, _, files in os.walk(root_dir):
        if filename in files:
            return True
    return False

We can also write a function that we'll use to run executables from within the Notebook.

In [8]:
import subprocess

def MatEnsemble_submitter(cmd_args, working_directory):
    try:
        subprocess.run(
            cmd_args,
            cwd=working_directory,
            check=True,              # <-- THIS is the key
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,  # capture error output
            text=True
        )
    except subprocess.CalledProcessError as e:
        if e.returncode == 1:
            print(
                "Workflow exited with code 1.\n"
                "This usually means no remaining jobs were found.\n"
                "Check whether the workflow has already converged."
            )
        else:
            print("Workflow failed with unexpected error:")
            print(e.stderr)

### a. Query Materials Project Bulk Structures

We will begin by creating a subdirectory for our VASP DFT calculations. 

In [9]:
dft_path = os.path.abspath(os.path.join(os.getcwd(), 'DFT'))
os.makedirs(dft_path, exist_ok=True)

Ensemble-FF-Fit has several command line executables installed that can be used to generate structures for DFT runs. These executables write and modify ```POSCAR``` files, which is the VASP default.

The ```mp_query``` executable pulls structures from the [Materials Project Database](https://next-gen.materialsproject.org) based on a provided YAML directive file. Let's write a YAML file that only queries experimentally-relevant DFT structures for this system, which are indicated by a star next to their entry on the MP website. 

In [10]:
query_dct = {'mpids': ['mp-21765',
        'mp-7818',  
        'mp-2503', 
        'mp-21165', 
        'mp-2418',
        'mp-571383', 
        'mp-618', 
        'mp-568593']}

query_yaml = os.path.join(dft_path, 'PdSe.yml')
write_yaml(query_yaml, query_dct)

We can now generate ```POSCAR``` files for our DFT runs. We will call this, and other, executables in this notebook using ```subprocess.run()```.

In [11]:
path_to_bulk_poscars = os.path.join(dft_path, 'structures/bulk')
os.makedirs(path_to_bulk_poscars, exist_ok=True)

subprocess.run(["mp_query",
                "--poscars_directory", path_to_bulk_poscars,
                "--max_atoms", "100",
                "--MP_yaml", query_yaml], stdout=subprocess.DEVNULL)

Retrieving SummaryDoc documents: 100%|██████████| 8/8 [00:00<00:00, 296941.88it/s]


CompletedProcess(args=['mp_query', '--poscars_directory', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk', '--max_atoms', '100', '--MP_yaml', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/PdSe.yml'], returncode=0)

These bulk structures provide a starting point for our fine-tuning dataset, but will need to be optimized with the DFT input parameters we'd like to use. Running this calculations will provide hands-on exprerience with MatEnsemble. 

### b. Optimizing Bulk Structures
Let's first create the YAML directive file and template submission script required by [vaspFlux](https://code.ornl.gov/rym/vaspflux) to create the inputs for VASP DFT jobs. We will set VASP parameters consistently across all calculations in this Notebook. 

In [12]:
input_path = os.path.join(dft_path, 'inputs_directory')
os.makedirs(input_path, exist_ok=True)

opt_dct = {'user_incar_settings': {'ALGO': "Normal",
                                   'EDIFF': 1.0e-05, 
                                   'EDIFFG': -0.01, 
                                   'ISIF': 2, 
                                   'NPAR': 16, 
                                   'KPAR': 4, 
                                   'ISMEAR': 0, 
                                   'SIGMA': 0.03, 
                                   'IVDW': 12,
                                   'NSW': 100, 
                                   'SYMPREC': 1.0e-06}, 
           'user_kpoints_settings': {'grid_density': 1000}}

opt_yaml_path = os.path.join(input_path, 'vdW_optimize.yml')
write_yaml(opt_yaml_path, opt_dct)           

vaspFlux's ```generate_vaspflux``` executable takes a template job submission file used to run individual VASP jobs on HPC. We provide an example template script, with GPU support, below.    

vaspFlux's ```submit_matsemble``` executable currently reads the number of individual tasks from this submission script, though there are plans to deprecate this behavior in the future.  

In [13]:
perlmutter_vasp_submission_string = '''#!/bin/bash
#
# Change to your account
# Also change in the srun command below
#SBATCH -A {ALLOCATION}
#
# Job naming stuff
#SBATCH -J {JOB_NAME}
#SBATCH -o %x-%j.out
#SBATCH -e %x-%j.err
#
# Requested time
#SBATCH -t {TIME}
#
# Requested queue
#SBATCH -C gpu
#SBATCH -q {PARTITION}
#
# Number of perlmutter nodes to use.
# Set the same value in the SBATCH line and NNODES
#SBATCH -N {NODES}

#SBATCH --exclusive
NNODES={NODES}

GPUS_PER_NODE={GPUS_PER_NODE}
GPUS_PER_TASK={GPUS_PER_TASK}
NGPUS=$(($GPUS_PER_NODE * $NNODES))

CPUS_PER_NODE={CPUS_PER_NODE}
CPUS_PER_TASK={CPUS_PER_TASK}
NCPUS=$(($CPUS_PER_NODE * $NNODES))

NUM_TASKS=$(($NGPUS / $GPUS_PER_TASK))

export OMP_NUM_THREADS=1
export OMP_PLACES=threads
export OMP_PROC_BIND=spread

module load vasp/6.4.3-gpu # Or version to be loaded\n

VASP_BINARY={VASP_EXECUTABLE}

# Run command. Modify to where ever you placed the binary and input files
srun -A m2113_g --ntasks=$NUM_TASKS --cpus-per-task $CPUS_PER_TASK --cpu-bind=cores --gpu-bind=none --gpus=$NGPUS $VASP_BINARY'''

perlmutter_vasp_submission_path = os.path.join(input_path, 'perlmutter_vasp_submission.sh')
write_string(perlmutter_vasp_submission_path, perlmutter_vasp_submission_string)

Now, let's call the ```generate_vaspflux``` executable to generate inputs for the queried `POSCAR` files. 

In [14]:
subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_bulk_poscars,
                    "--vasp_yaml", opt_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

CompletedProcess(args=['generate_vaspflux', '--poscars_directory', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk', '--vasp_yaml', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/inputs_directory/vdW_optimize.yml', '--vasp_submission', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/inputs_directory/perlmutter_vasp_submission.sh', '--atoms_per_node', '80'], returncode=0)

We can now run ```submit_matsemble``` to submit and run these calculations. **Note** most of the MatEnsemble-supported executables shown in this Notebook contain a `--dry_run` flag. This can help scope the number of calculations to be run, and what resources they require. 

In [15]:
submit_bulk_MatEnsemble = False

dft_outputs_path = os.path.join(dft_path, 'outputs')
dft_bulk_outputs_path = os.path.join(dft_outputs_path, 'bulk')
os.makedirs(dft_bulk_outputs_path, exist_ok=True)

if submit_bulk_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_bulk_poscars, "--dry_run"])    
else:
    bulk_dft_cmd_args = ["srun",
                "--ntasks-per-node=1",
                "--gpus-per-task=4",
                "flux", "start",
                "matsemble_vaspflux",
                "--parent_directory", 
                path_to_bulk_poscars]
    
    MatEnsemble_submitter(bulk_dft_cmd_args, dft_bulk_outputs_path)

No files to run; exiting workflow


### c. Generating defect structures
After finishing these bulk optimizations, we can copy the final structures to new directories and modify them to perform additional calculations. 

For example, let's create a `defect` directory and copy the `CONTCAR` files present in the `bulk` directory tree to `POSCAR` files in this new directory. This is conveniently performed with an the ```submit_vaspflux``` executable provided in vaspFlux. 

In [16]:
path_to_defect_poscars = os.path.join(dft_path, 'structures/defects')

if not os.path.isdir(path_to_defect_poscars):
    subprocess.run(["submit_vaspflux",
                "--parent_directory", path_to_bulk_poscars,
                "--move", "--move_to", path_to_defect_poscars], stdout=subprocess.DEVNULL)

Additionally, we can generate defective structures using the in-build ```generate_defects``` executable. Currently, we are passing the "-v" flag to generate only vacancies, but we encourage the user to explore the different possible defects that can be generated. 

**Note:** Larger structures can take a very long time to enumerate all defects (more unique defects to enumerate), so a conservative first pass is often the most prudent. 

In [17]:
if not file_exists_anywhere(path_to_defect_poscars, 'POSCAR.vasp'):
    subprocess.run(["generate_defects",
                "--poscars_directory", path_to_defect_poscars, 
                "-v"], stdout=subprocess.DEVNULL)

We will also write an input YAML for VASP single points, using many of the same input parameters as before.

In [18]:
from copy import deepcopy

sp_dct = deepcopy(opt_dct)
sp_dct['user_incar_settings']['NSW'] = 0

sp_yaml_path = os.path.join(input_path, 'vdW_single_point.yml')
write_yaml(sp_yaml_path, sp_dct)

Now, let's generate the inputs and run the VASP workflow.

In [19]:
# Create the defect calculation inputs
subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_defect_poscars,
                    "--vasp_yaml", sp_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

submit_defects_MatEnsemble = False
dft_defects_outputs_path = os.path.join(dft_outputs_path, 'defects')
os.makedirs(dft_defects_outputs_path, exist_ok=True)

# Run the defects with MatEnsemble
if submit_defects_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_defect_poscars, "--dry_run"])   
else:
    defects_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_defect_poscars]
    MatEnsemble_submitter(defects_dft_cmd_args, dft_defects_outputs_path)

No files to run; exiting workflow


### d. Rescaling structures 
We can run ```generate_EoS``` on the bulk structures to rescale them isotropically, preserving symmetries. 

In [20]:
path_to_rescaled_poscars = os.path.join(dft_path, 'structures/EoS')

if not os.path.isdir(path_to_rescaled_poscars):
    subprocess.run(["submit_vaspflux",
                "--parent_directory", path_to_bulk_poscars,
                "--move", "--move_to", path_to_rescaled_poscars], stdout=subprocess.DEVNULL)

if not file_exists_anywhere(path_to_rescaled_poscars, 'POSCAR.vasp'):
    subprocess.run(["generate_EoS",
                "--poscars_directory", path_to_rescaled_poscars], stdout=subprocess.DEVNULL)

submit_EoS_MatEnsemble = False

subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_rescaled_poscars,
                    "--vasp_yaml", sp_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

dft_EoS_outputs_path = os.path.join(dft_outputs_path, 'EoS')
os.makedirs(dft_EoS_outputs_path, exist_ok=True)

if submit_EoS_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_rescaled_poscars, "--dry_run"]) 
else:
    EoS_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_rescaled_poscars]
    MatEnsemble_submitter(EoS_dft_cmd_args, dft_EoS_outputs_path)

No files to run; exiting workflow


This and other functionality present in Ensemble-FF-Fit, but not shown here, provides simple ways to create a starting dataset with limited user intervention. 

### e. Isolated Atom Energies

We also need to generate isolated atomic energies at this level of theory for the MACE force field; we'll do that now. 

In [21]:
path_to_isolated_poscars = os.path.join(dft_path, 'structures/atomic')

elements = ['Pd', 'Se']
paths_to_isolated_atoms_elements = [os.path.join(path_to_isolated_poscars, el) for el in elements]

from pymatgen.core.structure import Structure

lattice = [[20, 0, 0], [0, 20, 0], [0, 0, 20]]
coords = [[10, 10, 10]]

for i, element in enumerate(elements): 
    structure =  Structure(lattice=lattice, 
                           species=[element], 
                           coords=coords, 
                           coords_are_cartesian=True)
    os.makedirs(paths_to_isolated_atoms_elements[i], exist_ok=True)
    structure.to(os.path.join(paths_to_isolated_atoms_elements[i], 'POSCAR'), fmt='poscar')

In [22]:
submit_atomic_MatEnsemble = False

subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_isolated_poscars,
                    "--vasp_yaml", sp_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

dft_atomic_outputs_path = os.path.join(dft_outputs_path, 'atomic')
os.makedirs(dft_atomic_outputs_path, exist_ok=True)

if submit_atomic_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_isolated_poscars, "--dry_run"]) 
else:
    atomic_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_isolated_poscars]
    MatEnsemble_submitter(atomic_dft_cmd_args, dft_atomic_outputs_path)

No files to run; exiting workflow


We'll parse these isolated atom energies into a dictionary of {atomic_number: final_energy} for force-field fitting.

In [23]:
from pymatgen.io.vasp.outputs import Vasprun

def single_atom_energies(path):
    dct = {}
    for root, _, _ in os.walk(path):
        vasprun_path = os.path.join(root, 'vasprun.xml')
        if os.path.exists(vasprun_path):
            v = Vasprun(vasprun_path)
            number = v.structures[-1][0].specie.number
            energy = v.final_energy
            dct[number] = energy
    return dct

atomic_dct = single_atom_energies(path_to_isolated_poscars)

## 3. Parse DFT Calculations

Here, we will demonstrate the ```parse2fit``` function call that will allow us to parse the raw VASP outputs into an extended .xyz file for MACE force field fitting. 

We will start by writing a generic YAML file to guide the parsing (see ```parse2fit``` documentation for formatting). **Note**: Don't worry about the errors with weighting that are output; parse2fit was originally designed for more complicated ReaxFF data formatting. For MACE, we just need  extended.xyz files. 

In [24]:
data_directory = os.path.abspath(os.path.join(os.getcwd(), 'Data'))
initial_data_directory = os.path.join(data_directory, 'initial')
os.makedirs(data_directory, exist_ok=True)

label_name = 'Bulk'
parse2fit_string = f"""# extended .xyz Layout

output_format: 'fitsnap' # consistent tag across all .yaml

generation_parameters:
  runs_to_generate: 1
  #input_directory: # Not supported
  #fix_input_weights: False # Not supported
  output_directory: {initial_data_directory}

default_weights: # Default is 50/50 0 or 1
  energy:
    type: 'normal'
    min: 0.075
    max: 5.0
    sigma: 3.0
    scale: 10
  charges:
    type: 'binary'
    min: 0.0
    max: 0.001
    split: 0.7

method_parameters:
  scraper:
    scraper: 'xyz'
  path:
    dataPath: 'XYZ'

input_paths:
  # Set a path here
  #
  {label_name}:
    directories:
      - {path_to_bulk_poscars}
    energy: True
    forces: True
    lattice_vectors: True
    stress_vectors: True"""

parse2fit_path = os.path.join(data_directory, 'p2f_example.yml')
write_string(parse2fit_path, parse2fit_string)

Let's set a variable to the path of the .xyz file to write and delete it if it already exists. Then, let's run ```generate_parse2fit``` on these VASP directories to create example training data from the VASP input files. 

In [25]:
example_xyz_path = os.path.join(initial_data_directory, f'fitsnap_run_0/XYZ/{label_name}.xyz')
if os.path.exists(example_xyz_path):
    os.remove(example_xyz_path) # Avoid appending the data
subprocess.run(["generate_parse2fit", parse2fit_path])

Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/Pd4Se/mp-7818; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/PdSe2/mp-2418; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/Pd7Se2/mp-618; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/PdSe/mp-21165; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/Pd7Se4/mp-2503; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/Pd17Se15/mp-21765; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT/structures/bulk/Pd-Se/PdSe/mp-571383; parsing
Electronically converged vasprun.xml in /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/DFT

CompletedProcess(args=['generate_parse2fit', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/Data/p2f_example.yml'], returncode=0)

The file should show structures run with DFT and parsed for fitting, which we can quickly check with our ```read_file``` function.

In [26]:
read_file(example_xyz_path)[0:12]

['10\n',
 'Lattice="5.28074265 0.0 0.0 0.0 5.28074265 0.0 0.0 0.0 5.68863415" Properties=species:S:1:pos:R:3:forces:R:3 energy=-56.2770567 stress="-33804.639129999996 -0.0 -0.0 -0.0 -33804.639129999996 -0.0 -0.0 -0.0 -34767.95017" \n',
 'Pd 4.053890575741174 1.9668996794387983 4.811182242582025 0.00808942 0.00200573 -0.00154346\n',
 'Pd 4.607271004438798 1.4135192507411745 1.966865167582025 0.00200573 0.00808942 -0.00154346\n',
 'Pd 0.6734716455612014 3.8672233992588256 1.966865167582025 -0.00200573 -0.00808942 -0.00154346\n',
 'Pd 3.3138429705612014 4.053890575741174 0.8774519074179751 -0.00200573 0.00808942 0.00154346\n',
 'Pd 1.4135192507411745 0.6734716455612014 3.721768982417975 0.00808942 -0.00200573 0.00154346\n',
 'Pd 3.8672233992588256 4.607271004438798 3.721768982417975 -0.00808942 0.00200573 0.00154346\n',
 'Pd 1.2268520742588256 3.3138429705612014 4.811182242582025 -0.00808942 -0.00200573 -0.00154346\n',
 'Pd 1.9668996794387983 1.2268520742588256 0.8774519074179751 0.002005

We now have data formatted for MACE force field fitting! 

By changing which data is passed in the input YAML directive, parse2fit can be used to generate different combinations of training data, further influencing the ensemble force field fit. 

Varying the training data in this manner can be useful to quickly assess (1) which data is most beneficial for training, and (2) how variable the resulting force fields become when subsets of the full training set are used. 

## 4. Fit an Ensemble of Force Fields

So far, we've explored different ways to generate ~100 unique structures for MACE force field fitting and evaluate their properties with DFT, most of which are near-equilibrium. This limited data is likely insufficient to reliably fine-tune a foundation model. 

Therefore, to fit the following force field ensemble, we will use pre-parsed DFT data: a ```train.xyz``` file  for our train/test split and a ```test.xyz``` file for validation. These include structures with greater diversity that were generated with tools in ```Ensemble-FF-Fit```, but require greater user intervention to construct. 

In [27]:
ff_path = os.path.join(os.getcwd(), 'FF')
ff_inputs_path = os.path.join(ff_path, 'inputs_directory/thermal_subbed_EoS')

Now that we have the data, our next step will be to create the input files that dictate how the force field is fit. 

We can use a generic dictionary template for this fitting, and write a simple function which reads this template, makes modifications, and writes out ```config.yml``` files to subdirectories located in the ```inputs_directory```. 

In [28]:
freeze = [6, 7]
weights = [(1.0, 20.0, 100.0), (1.0, 20.0, 50.0), (1.0, 20.0, 20.0)]

base_MACE_dct = {'energy_key': 'energy', 
                 'forces_key': 'forces', 
                 'stress_key': 'stress', 
                 'device': 'cuda', 
                 'batch_size': 10, 
                 'max_num_epochs': 100,
                 'num_samples_pt': 300,
                 'swa': False,
                 'model': 'MACE',
                 'stress_weight': 1.0,
                 'forces_weight': 20.0,
                 'energy_weight': 100.0,
                 'freeze': 6,
                 'E0s': atomic_dct, # {34: -0.77605851, 46: -1.4757099},
                 'multiheads_finetuning': False,
                 'valid_fraction': 0.15,
                 'enable_cueq': False}

def write_MACE_YAMLs(default_FF_yaml, write_path, freeze, weights):
    for f in freeze:
        f_dir = os.path.join(write_path, f'freeze_{str(f)}')
        #os.makedirs(f_dir, exist_ok=True)
        for i, weight in enumerate(weights):
            w_dir = os.path.join(f_dir, f'energy_{str(i+1)}')
            os.makedirs(w_dir, exist_ok=True)
            base_MACE_dct['freeze'] = f
            base_MACE_dct.update({'stress_weight': weight[0], 'forces_weight': weight[1], 'energy_weight': weight[2]})
            write_yaml(os.path.join(w_dir, 'config.yml'), base_MACE_dct)
    return 

write_MACE_YAMLs(base_MACE_dct, ff_inputs_path, freeze, weights)

Let's next pull and format the MACE model to be used for our fine-tuning. 

We'll refit the MACE-OMAT-medium foundation model, which can be queried using the [MACE](https://github.com/ACEsuit/mace) Python interface; we can write this to an appropriate directory with a subprocess call. This is necessary in the current Notebook setup, because the default Python supports the Spack environment containing Flux, so we must set the Python executable to the environment's Python. 

In [29]:
import torch
from mace.calculators import mace_mp

model_name = 'medium-omat-0'
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load a passed foundation model
model = mace_mp(
    model=model_name,  # or any listed foundation model
    device=device,
    return_raw_model=True
)

mace_model_write_directory = os.path.join(ff_path, model_name)

os.makedirs(mace_model_write_directory, exist_ok=True)
torch.save(model, os.path.join(mace_model_write_directory, "model.model"))

/global/homes/r/rym/.conda/envs/unified/lib/python3.11/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using model under Academic Software License (ASL) license, see https://github.com/gabor1/ASL 
 To use this model you accept the terms of the license.
Using Materials Project MACE for MACECalculator with /global/homes/r/rym/.cache/mace/maceomat0mediummodel
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/global/homes/r/rym/.conda/envs/unified/lib/python3.11/site-packages/mace/calculators/foundations_models.py:154: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  return torch.load(model_path, map_location=device)


We can now fit an ensemble of force fields using the executable provided in Ensemble-FF-Fit. 

In [30]:
submit_MatEnsemble = False

ff_fitting_cmd_args = ["mace_matensemble",
        "--inputs_directory", ff_inputs_path, 
        "--run_directory", mace_model_write_directory, 
        "--train_file", 'train.xyz', 
        "--test_file", 'test.xyz', 
        "--fits_per_runpath", '1', 
        "--foundation_model", "model.model"]

if submit_MatEnsemble is False:
    subprocess.run(ff_fitting_cmd_args + ["--dry_run"]) 
else:
    MatEnsemble_submitter(["srun", "flux", "start"] + ff_fitting_cmd_args, 
                          dft_EoS_outputs_path)

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_6/energy_3/6995, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_6/energy_2/6781, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_6/energy_1/3248, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_7/energy_3/1076, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_7/energy_2/9381, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0/thermal_subbed_EoS/freeze_7/energy_1/7072, tasks: 1

Total tasks = 6; cpus_per_task=16; gpus_per_task=1


# A. Fitting an Ensemble of Force Fields on HPC

## 1. Run Pd-Se Melting Simulations

### a. Finite Temperature MD

Now we are at the main part of fine-tuning; selecting the new structures to add to the dataset and determining the best force field to spawn the next generation's ensemble. 

We will start by randomly selecting structures from our VASP runs to be used a starting structures for finite-temperature MD runs. We will select structures from groups of parent structures to ensure greater variability in our selection. 

In [31]:
md_directory = os.path.abspath(os.path.join(os.getcwd(), 'MD'))
md_FT_directory = os.path.join(md_directory, 'FT')
md_FT_inputs_directory = os.path.join(md_FT_directory, 'inputs_directory')
os.makedirs(md_FT_inputs_directory, exist_ok=True)

In [32]:
import random
from collections import defaultdict
import re

def get_file_paths(path, check_file='POSCAR'):
    all_paths = []
    for root, _, _ in os.walk(path):
        use_file = os.path.join(root, check_file)
        if os.path.exists(use_file):
            all_paths.append(root)
    return all_paths
     
def group_paths_by_mpid(paths):
    groups = defaultdict(list)
    for p in paths:
        try:
            mpid = re.search(r"mp-(\d+)", p).group(0)
            groups[mpid].append(p)
        except AttributeError:
            continue
    return groups

root_paths = get_file_paths(dft_path, check_file='CONTCAR')
root_paths_dct = group_paths_by_mpid(root_paths)
randomly_sampled = [random.choice(v) for v in root_paths_dct.values()]

## Now move to the inputs directory
copy_sampled = False
if copy_sampled:
    for sampled_path in randomly_sampled:
        subprocess.run(["submit_vaspflux",
                "--parent_directory", sampled_path,
                "--move", "--move_to", md_inputs_directory], stdout=subprocess.DEVNULL)

We now have starting structures for our MD runs; we can write a Python script that uses the ASE package to run a finite-temperature MD protocol with MatEnsemble.

In [33]:
ase_mace_ft_string = """import sys
import os
import torch
import gc
from mace.calculators import MACECalculator
from EnsembleFFFit.matensemble.lammps.helpers import parse_list
from ase.io import read
from ase.io import Trajectory
from ase.md.langevin import Langevin
#from ase.md.verlet import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
import ase.units as units
import json

if __name__ == "__main__":

    ff_list       = parse_list(sys.argv[1]) # Force field file paths
    input_list    = parse_list(sys.argv[2]) # Input file paths
    struct_list   = parse_list(sys.argv[3]) # Structure file paths
    output_list   = parse_list(sys.argv[4]) # LAMMPs output write paths

    n = len(ff_list)
    assert all(len(lst) == n for lst in (input_list, struct_list, output_list)), "All lists must be same length"

    for idx, (ff, inp, struct, output) in enumerate(zip(ff_list, input_list, struct_list, output_list), start=0):
        
        # 1a) Load the MACE model
        calculator = MACECalculator(model_path=ff, device="cuda" if torch.cuda.is_available() else "cpu")

        # 1b) Load the structure file
        init_conf = read(struct)

        # 2) Initialize the calculation
        init_conf.set_calculator(calculator)
        with open(inp) as fh: 
            cfg = json.load(fh)
        MaxwellBoltzmannDistribution(init_conf, temperature_K=cfg['temperature'])

        # --- MD integrator
        dt = cfg['timestep'] * units.fs
        friction = cfg['friction'] / units.fs
        #dyn = VelocityVerlet(init_conf, dt)
        
        dyn = Langevin(atoms=init_conf, 
                       timestep=dt, 
                       temperature_K=cfg['temperature'], 
                       friction=friction)

        # 3) Run the MD and update the status of property_dictionary
        property_dict = {}

        def update_status():
            step = dyn.get_number_of_steps()
            energy = init_conf.get_potential_energy()
            forces = init_conf.get_forces()
            property_dictionary = {
                'energy': float(energy),
                'fx': [float(f[0]) for f in forces],
                'fy': [float(f[1]) for f in forces],
                'fz': [float(f[2]) for f in forces],
            }
            property_dict[step] = property_dictionary

        # attach the callback to run every 1000 steps
        dyn.attach(update_status, cfg['frequency'])

        # --- trajectory and attachments
        traj = Trajectory(os.path.join(output, 'md_run.traj'), 'w', init_conf)
        dyn.attach(traj.write, cfg['frequency'], init_conf)

        # Run the MD
        dyn.run(cfg['nsteps'])

        # close trajectory file
        traj.close()

        # --- after run: write the property_dict to disk once ---
        output_name = os.path.join(output, 'properties.json')
        with open(output_name, "w") as f:
            json.dump(property_dict, f, indent=4)

        # --- after run: free Python memory ---
        if torch.cuda.is_available():
            del calculator, init_conf, dyn  # remove large objects
            gc.collect()                # free Python memory
            torch.cuda.empty_cache()    # release unreferenced GPU memory back to CUDA driver"""

ase_mace_ft_path = os.path.join(md_FT_inputs_directory, 'ase_mace.py')
write_string(ase_mace_ft_path, ase_mace_ft_string)

This script requires a protocol json file to run the molecular dynamics. Let's use a temperature of 1000 K to thermalize these cells. 

In [34]:
from copy import deepcopy

MD_1000_path = os.path.join(md_FT_inputs_directory, '1000K')
os.makedirs(MD_1000_path, exist_ok=True)
MD_1000_json_path = os.path.join(MD_1000_path, 'in.mace.json')

frequency = 1000
timestep = 1.0

MD_dct_1000 = {
  "temperature": 1000,
  "nsteps": 15000,
  "frequency": frequency,
  "friction": 0.01,
  "timestep": timestep
}
write_json(MD_1000_json_path, MD_dct_1000)

#MD_2000_path = os.path.join(md_FT_inputs_directory, '2000K')
#os.makedirs(MD_2000_path, exist_ok=True)
#MD_2000_json_path = os.path.join(MD_2000_path, 'in.mace.json')

#MD_dct_2000 = deepcopy(MD_dct_1000)
#MD_dct_2000['temperature'] = 2000
#write_json(MD_2000_json_path, MD_dct_2000)

Next, let's copy the default MACE model we used for our ensemble fine-tuning to a directory inside the FT MD directory. 

In [35]:
md_FT_MACE_directory = os.path.join(md_FT_directory, model_name)
os.makedirs(md_FT_inputs_directory, exist_ok=True)
copy_files(mace_model_write_directory, md_FT_MACE_directory, patterns=("model.model"))

We can now launch the molecular dynamics simulations using our MatEnsemble call. 

In [36]:
md_ft_outputs_directory = os.path.join(md_FT_directory, 'outputs')
md_ft_outputs_iter_directory = os.path.join(md_ft_outputs_directory, '0')
os.makedirs(md_ft_outputs_iter_directory, exist_ok=True)
model_string = "model.model"

submit_MatEnsemble = False

ft_md_cmd_args = ["lammps_matensemble",
                  "--run_directory", md_FT_MACE_directory,
                  "--inputs_directory", md_FT_inputs_directory,
                  "--ffield", model_string,
                  "--structure", "POSCAR",
                  "--control", "None",
                  "--in_lammps", "in.mace.json",
                  "--lammps_task", "ase_mace.py",
                  "--atoms_per_task", "1000",
                  "--lammps_task_order", "ffield", "in_lammps", "structure", 
                  "--parent_levels", "0",
                  "--gpus_per_task", "1",
                  "--cpus_per_task", "16",
                  "--atom_style", "atomic"]

if submit_MatEnsemble is False:
    subprocess.run(ft_md_cmd_args + ["--dry_run"]) 
else:
    MatEnsemble_submitter(["srun", "flux", "start"] + ft_md_cmd_args, 
                          md_ft_outputs_iter_directory)

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/bulk/Pd-Se/PdSe/mp-21165, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/defects/Pd-Se/Pd4Se/mp-7818/vacancy/vacancy0, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/defects/Pd-Se/Pd17Se15/mp-21765/vacancy/vacancy0, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/defects/Pd-Se/PdSe/mp-571383/vacancy/vacancy4, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/EoS/Pd-Se/Pd7Se4/mp-2503/volume_1.15, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/EoS/Pd-Se/PdSe2/mp-2418/volume_0.85, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/FT/medium-omat-0/structures/EoS/Pd-Se/

### b. Single Points

We will now create a directory where we run single point calculations using our ensemble of force fields for all structures generated from the MD simulations.  

In [37]:
md_SP_directory = os.path.join(md_directory, 'SP')
md_SP_inputs_directory = os.path.join(md_SP_directory, 'inputs_directory')
os.makedirs(md_SP_inputs_directory, exist_ok=True)

First, let's write POSCAR files from the different images along the MD trajectories. 

**Note:** We'll make sure that the directory naming is consistent between the FT and SP runs. Consistent naming conventions will come in handy later, when we are directly comparing between multiple MD runs and DFT ground truth properties. 

In [38]:
from ase.io.trajectory import Trajectory
from pymatgen.io.ase import AseAtomsAdaptor

def get_trajectory_paths(path, traj_name="md_run.traj"):
    all_paths = []
    for root, _, _ in os.walk(path):
        traj_path = os.path.join(root, "md_run.traj")
        if os.path.exists(traj_path):
            all_paths.append(traj_path)
    return all_paths

def write_trajectories_to_POSCARs(to_path, split_path, all_paths):
    aaa = AseAtomsAdaptor()
    
    for path in all_paths:
        base_path = path.split(split_path)[-1][1:] # Strip leading character
        write_to_path = os.path.dirname(os.path.join(to_path, base_path))
        os.makedirs(os.path.dirname(write_to_path), exist_ok=True)
        traj = Trajectory(path)
        for i, ase_atoms in enumerate(traj):
            structure = aaa.get_structure(ase_atoms)
            write_to_path_image = os.path.join(write_to_path, str(int(i * frequency * timestep))) # By number of femtoseconds
            os.makedirs(write_to_path_image, exist_ok=True)
            structure.to(fmt='poscar', filename=os.path.join(write_to_path_image, 'POSCAR'))        
    return 

trajectory_paths = get_trajectory_paths(md_FT_MACE_directory)
write_trajectories_to_POSCARs(md_SP_inputs_directory, md_FT_MACE_directory, trajectory_paths)

Next, we'll write the input files to run single point calculations with ASE. 

In [39]:
ase_mace_sp_string = """import sys
import os
import torch
from mace.calculators import MACECalculator
from EnsembleFFFit.matensemble.lammps.helpers import parse_list
from ase.io import read, write
import json

if __name__ == "__main__":
    ff_list       = parse_list(sys.argv[1]) # Force field file paths
    struct_list   = parse_list(sys.argv[2]) # Structure file paths
    output_list   = parse_list(sys.argv[3]) # LAMMPs output write paths

    n = len(ff_list)
    assert all(len(lst) == n for lst in (struct_list, output_list)), "All lists must be same length"

    for idx, (ff, struct, output) in enumerate(zip(ff_list, struct_list, output_list), start=0):
        property_dictionary = {}

        # 1a) Load the MACE model
        calculator = MACECalculator(model_path=ff, device="cuda" if torch.cuda.is_available() else "cpu")

        # 1b) Load the structure file
        init_conf = read(struct)

        # 1c) Write the structure file
        write(filename=os.path.join(output, 'POSCAR'), images=init_conf)

        # 2) Compute the potential energy
        init_conf.set_calculator(calculator)
        energy = init_conf.get_potential_energy()
        forces = init_conf.get_forces()

        # 3) Write the energy to the output path
        property_dictionary['energy'] = energy
        property_dictionary['fx'] = [forces[i][0] for i in range(len(forces))]
        property_dictionary['fy'] = [forces[i][1] for i in range(len(forces))]
        property_dictionary['fz'] = [forces[i][2] for i in range(len(forces))]

        with open(os.path.join(output, 'properties.json'), "w") as f:
            json.dump(property_dictionary, f, indent=4)"""

ase_mace_sp_path = os.path.join(md_SP_inputs_directory, 'ase_mace.py')
write_string(ase_mace_sp_path, ase_mace_sp_string)

Finally, we'll copy the fitted force fields in a format so that MatEnsemble can run the collection of predictions. 

In [40]:
md_SP_MACE_directory = os.path.join(md_SP_directory, model_name)
os.makedirs(md_SP_inputs_directory, exist_ok=True)

# Then copy over all files
ffield_pattern = '^MACE_([^._-]+)\\.model$' # Regex pattern for output MACE files

subprocess.run(["copy_by_pattern",
                    "--source_directory", mace_model_write_directory,
                    "--target_directory", md_SP_MACE_directory,
                    "--target_name", model_string, 
                    "--pattern", ffield_pattern], stdout=subprocess.DEVNULL)

CompletedProcess(args=['copy_by_pattern', '--source_directory', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/FF/medium-omat-0', '--target_directory', '/pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0', '--target_name', 'model.model', '--pattern', '^MACE_([^._-]+)\\.model$'], returncode=0)

Now, we can submit the full workflow using MatEnsemble to compute single point energies. 

**Note**: We pass --parents_levels=1 to reduce latency time for the calculation. This groups the single point runs on the same GPU, instead of re-initializing a new single point for each calculation. 

In [41]:
md_sp_outputs_directory = os.path.join(md_SP_directory, 'outputs')
md_sp_outputs_iter_directory = os.path.join(md_sp_outputs_directory, '0')
os.makedirs(md_sp_outputs_iter_directory, exist_ok=True)

submit_MatEnsemble = False

sp_md_cmd_args = ["lammps_matensemble",
                  "--run_directory", md_SP_MACE_directory,
                  "--inputs_directory", md_SP_inputs_directory,
                  "--ffield", model_string,
                  "--structure", "POSCAR",
                  "--control", "None",
                  "--in_lammps", "None",
                  "--lammps_task", "ase_mace.py",
                  "--atoms_per_task", "1000",
                  "--lammps_task_order", "ffield", "structure", 
                  "--parent_levels", "1",
                  "--gpus_per_task", "1",
                  "--cpus_per_task", "16",
                  "--atom_style", "atomic"]

if submit_MatEnsemble is False:
    subprocess.run(sp_md_cmd_args + ["--dry_run"]) 
else:
    MatEnsemble_submitter(["srun", "flux", "start"] + sp_md_cmd_args, 
                          md_sp_outputs_iter_directory)

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/bulk/Pd-Se/PdSe/mp-21165, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/defects/Pd-Se/Pd4Se/mp-7818/vacancy/vacancy0, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/defects/Pd-Se/Pd17Se15/mp-21765/vacancy/vacancy0, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/defects/Pd-Se/PdSe/mp-571383/vacancy/vacancy4, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medium-omat-0/freeze_6/energy_3/314/314/structures/EoS/Pd-Se/Pd7Se4/mp-2503/volume_1.15, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/SP/medi

## 2. Determine the images with the greatest uncertainty

Now that we have run single points for the force field ensemble on the collection of structures, we can write some code that chooses the structures where force and energy variance is largest for the newest DFT runs.

We can do this in a "hacky" way with some imports available in Ensemble-FF-Fit, but we will have to use subprocess to call a code snippet that can retrieve the sorted structures for us, based on variance scoring. 

In [42]:
variance_code = r"""
import sys, json
from EnsembleFFFit.analysis.dict_parsers import ASEParser
from EnsembleFFFit.analysis.variance import get_structures_scores

path = sys.argv[1]
label_tuple = eval(sys.argv[2])
energy_weight = float(sys.argv[3])
force_weight = float(sys.argv[4])

valid_parser = ASEParser(path)
valid_parsed_dct = valid_parser.parse_directory(label_tuple)

labels, images, structures, scores = get_structures_scores(
    valid_parsed_dct,
    energy_weight,
    force_weight,
)

out = {
    "labels": labels,
    "images": images,
    "scores": scores,
    "structures": [s.as_dict() for s in structures],
}

print(json.dumps(out))
"""

out = subprocess.check_output(
    [
        "/global/homes/r/rym/.conda/envs/unified/bin/python",
        "-c",
        variance_code,
        md_SP_MACE_directory,
        '(9, 14)', 
        '0.1', 
        '1.0'
    ],
    text=True
)

data = json.loads(out)

labels = data["labels"]
images = data["images"]
scores = data["scores"]
structures = [Structure.from_dict(d) for d in data["structures"]]

Let's look at the top 20 structures with the largest variance across the ensemble:

In [43]:
for i in range(1, 20):
    print(f'Label: {labels[i]}, Image: {images[i]},  Score: {scores[i]}')

Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 1000,  Score: 0.06002561688552383
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 2000,  Score: 0.05543450838293522
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 3000,  Score: 0.05521673246425243
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 12000,  Score: 0.04960340889952476
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 15000,  Score: 0.04950125992471358
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 13000,  Score: 0.04822843709800529
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 4000,  Score: 0.046981092147808684
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 10000,  Score: 0.045387599667812854
Label: structures/defects/Pd-Se/Pd34Se11/mp-568593/vacancy/vacancy10, Image: 7000,  Score: 0.0441071503589

Most likely, the most poorly predicted structures come from the same starting structure(s). 

We (likely) don't want to bias our training dataset by including different images from these same trajectories, so instead let's select the single image with the largest variance from each individual run, and write these images to a new folder for DFT calculations. 

In [44]:
def select_structures(labels, images, structures, scores, total=10, unique_run=True):
    if unique_run == True:
        count = 1
        selected_labels, selected_images, selected_structures, selected_scores = [], [], [], []
        for i, label in enumerate(labels):
            if label not in selected_labels and count < total:
                selected_labels.append(label)
                selected_images.append(images[i])
                selected_structures.append(structures[i])
                selected_scores.append(scores[i])
                count += 1
        return selected_labels, selected_images, selected_structures, selected_scores
    else:
        return labels[:total], images[:total], structures[:total], scores[:total]

def write_new_structures(to_dir, labels, images, structures):
    for i, structure in enumerate(structures):
        write_path = os.path.join(to_dir, os.path.join(labels[i], str(images[i])))
        os.makedirs(write_path, exist_ok=True)
        structure.to(
            fmt="poscar",
            filename=os.path.join(write_path, "POSCAR"),
        )
    return 

In [45]:
sel_labels, sel_images, sel_structures, sel_scores = select_structures(labels, images, structures, scores)
path_to_model_variance = os.path.join(dft_path, f'structures/variance/{model_name}')
write_new_structures(path_to_model_variance, sel_labels, sel_images, sel_structures)

## 3. Calculate DFT single-point energies

We can now regenerate single-point inputs for these structures and run them with DFT. 

In [46]:
subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_model_variance,
                    "--vasp_yaml", sp_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

dft_variance_outputs_path = os.path.join(dft_outputs_path, 'variance')
os.makedirs(dft_variance_outputs_path, exist_ok=True)

submit_MatEnsemble = False
if submit_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_model_variance, "--dry_run"])   
else:
    variance_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_model_variance]
    MatEnsemble_submitter(variance_dft_cmd_args, dft_variance_outputs_path)

No files to run; exiting workflow


## 4. Determine the best force field


Now that we have new data to add to our training, our final step is to select the "best" force field from our ensemble to use as the starting force field for the next generation. 

We will begin by computing DFT-level validation data, and then compare energy and force predictions across the ensemble to this validation data. 

**Note:** Choosing the best force field from validation data will depend upon the application at hand, and should be selected carefully. 

### a. AIMD Validation runs

We will begin by constructing an input YAML file for AIMD runs. 

In [47]:
aimd_dct = {'user_incar_settings': {'ALGO': "Normal",
                                   'EDIFF': 1.0e-05, 
                                   'EDIFFG': -0.01, 
                                   'ISIF': 3, 
                                   'NPAR': 16, 
                                   'KPAR': 4, 
                                   'ISMEAR': 0, #0 
                                   'SIGMA': 0.03, #0.03 
                                   'IVDW': 12,
                                   'NSW': 20, 
                                   'SYMPREC': 1.0e-06, 
                                   'IBRION': 0, 
                                   'POTIM': 2.0, 
                                   'TEBEG': 1500, 
                                   'MDALGO': 2, 
                                   'SMASS': -3}, 
           'user_kpoints_settings': {'grid_density': 1000}}

aimd_yaml_path = os.path.join(input_path, 'vdW_aimd.yml')
write_yaml(aimd_yaml_path, aimd_dct) 

And we will choose a the Pd-Se bulk structures to perform the AIMD on, starting from the structures that we optimized. 

In [48]:
path_to_aimd_poscars = os.path.join(dft_path, 'structures/aimd')
os.makedirs(path_to_aimd_poscars, exist_ok=True)

subprocess.run(["submit_vaspflux",
                "--parent_directory", path_to_bulk_poscars,
                "--move", "--move_to", path_to_aimd_poscars], stdout=subprocess.DEVNULL)

subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_aimd_poscars,
                    "--vasp_yaml", aimd_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "80"], stdout=subprocess.DEVNULL)

dft_AIMD_outputs_path = os.path.join(dft_outputs_path, 'AIMD')
os.makedirs(dft_AIMD_outputs_path, exist_ok=True)

submit_MatEnsemble = False
if submit_MatEnsemble is False:
    subprocess.run(["matsemble_vaspflux",
                "--parent_directory", path_to_aimd_poscars, "--dry_run"]) 
else:
    AIMD_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_aimd_poscars]
    MatEnsemble_submitter(AIMD_dft_cmd_args, dft_AIMD_outputs_path)

No files to run; exiting workflow


Let's write a simple function that pulls the images from the AIMD runs and writes them to a new folder of DFT calculations. 

In [49]:
from pymatgen.io.vasp import Vasprun

def diverging_subpaths(path1, path2):
    p1 = Path(path1).resolve().parts
    p2 = Path(path2).resolve().parts

    i = 0
    for a, b in zip(p1, p2):
        if a != b:
            break
        i += 1

    return (
        Path(*p1[i:]),  # diverging part of path1
        Path(*p2[i:])   # diverging part of path2
    )

def write_structures(structures, to_path):
    for i, structure in enumerate(structures):
        write_path = os.path.join(to_path, str(i))
        os.makedirs(write_path, exist_ok=True)
        structure.to(fmt='poscar', filename=os.path.join(write_path, 'POSCAR'))
    return 

def Vasprun_to_POSCARs(from_path, to_path):
    for root, _, _ in os.walk(from_path):
        vasprun_path = os.path.join(root, 'vasprun.xml')
        if os.path.exists(vasprun_path):
            try:
                v = Vasprun(vasprun_path)
                structures = v.structures
                full_with, first = diverging_subpaths(root, to_path)
                full = str(full_with).split("/", 1)[1]
                write_path = os.path.join(to_path, full)
                write_structures(structures, write_path)
            except:
                continue
    return 

path_to_aimd_sp_poscars = os.path.join(dft_path, 'structures/aimd_sp')
Vasprun_to_POSCARs(path_to_aimd_poscars, path_to_aimd_sp_poscars)

And now we'll run single points on these structures. 

In [50]:
subprocess.run(["generate_vaspflux",
                    "--poscars_directory", path_to_aimd_sp_poscars,
                    "--vasp_yaml", sp_yaml_path,
                    "--vasp_submission", perlmutter_vasp_submission_path, 
                    "--atoms_per_node", "100"], stdout=subprocess.DEVNULL)

dft_AIMD_sp_outputs_path = os.path.join(dft_outputs_path, 'AIMD_sp')
os.makedirs(dft_AIMD_sp_outputs_path, exist_ok=True)

submit_MatEnsemble = False
if submit_MatEnsemble is False:
    pass
    #subprocess.run(["matsemble_vaspflux",
    #            "--parent_directory", path_to_aimd_sp_poscars, "--dry_run"]) 
else:
    AIMD_sp_dft_cmd_args = ["srun",
                            "flux", "start",
                            "matsemble_vaspflux",
                            "--parent_directory", 
                            path_to_aimd_sp_poscars]
    MatEnsemble_submitter(AIMD_sp_dft_cmd_args, dft_AIMD_sp_outputs_path)

<string>:31: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affects the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.


### b. MD SP runs with Ensemble

Having converged the images from our AIMD validation set, we can now run FF predictions based on our Ensemble from this collection of structures. Let's first construct a Validation directory, with the appropriate inputs, to be used to run our collection of single points. 

In [51]:
# Create the directory 
md_Validation_directory = os.path.join(md_directory, 'Validation')
md_Validation_inputs_directory = os.path.join(md_Validation_directory, 'inputs_directory')
os.makedirs(md_Validation_inputs_directory, exist_ok=True)

# Create the outputs directory
md_Validation_outputs_directory = os.path.join(md_Validation_directory, 'outputs')
md_Validation_outputs_iter_directory = os.path.join(md_Validation_outputs_directory, '0')
os.makedirs(md_Validation_outputs_iter_directory, exist_ok=True)

# Write the appropriate files to the inputs_directory
ase_mace_Validation_path = os.path.join(md_Validation_inputs_directory, 'ase_mace.py')
write_string(ase_mace_Validation_path, ase_mace_sp_string)

# Move converged single-point structures to inputs_directory
subprocess.run(["submit_vaspflux",
                "--parent_directory", path_to_aimd_sp_poscars,
                "--move", "--move_to", md_Validation_inputs_directory], stdout=subprocess.DEVNULL)

# Copy over the force field ensemble with proper formatting
md_Validation_MACE_directory = os.path.join(md_Validation_directory, model_name)
os.makedirs(md_Validation_MACE_directory, exist_ok=True)

subprocess.run(["copy_by_pattern",
                    "--source_directory", mace_model_write_directory,
                    "--target_directory", md_Validation_MACE_directory,
                    "--target_name", 'model.model', 
                    "--pattern", ffield_pattern], stdout=subprocess.DEVNULL)

# Finally, submit the single-point runs
submit_MatEnsemble = False
valid_md_cmd_args = ["lammps_matensemble",
                  "--run_directory", md_Validation_MACE_directory,
                  "--inputs_directory", md_Validation_inputs_directory,
                  "--ffield", model_string,
                  "--structure", "POSCAR",
                  "--control", "None",
                  "--in_lammps", "None",
                  "--lammps_task", "ase_mace.py",
                  "--atoms_per_task", "1000",
                  "--lammps_task_order", "ffield", "structure", 
                  "--parent_levels", "1",
                  "--gpus_per_task", "1",
                  "--cpus_per_task", "16",
                  "--atom_style", "atomic"]

if submit_MatEnsemble is False:
    subprocess.run(valid_md_cmd_args + ["--dry_run"]) 
else:
    MatEnsemble_submitter(["srun", "flux", "start"] + valid_md_cmd_args, 
                          md_Validation_outputs_iter_directory)


path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/Pd4Se/mp-7818, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/Pd7Se4/mp-2503, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/PdSe2/mp-2418, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/Pd17Se15/mp-21765, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/PdSe/mp-21165, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_3/314/314/structures/aimd_sp/Pd-Se/PdSe/mp-571383, tasks: 1

path: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/ener

Let's parse these single points into a formatted dictionary, like we did previously with the other single points. We will write another short function to accomplish this through a subprocess call.

In [52]:
validation_code = r"""
import sys, json
from EnsembleFFFit.analysis.dict_parsers import ASEParser, VASPParser
from EnsembleFFFit.analysis.best_force_field import rank_ff_scores

md_path = sys.argv[1]
dft_path = sys.argv[2]
md_tuple = eval(sys.argv[3])
dft_tuple = eval(sys.argv[4])
energy_weight = float(sys.argv[5])
force_weight = float(sys.argv[6])

md_parser = ASEParser(md_path)
md_parsed_dct = md_parser.parse_directory(md_tuple)

dft_parser = VASPParser(dft_path)
dft_parsed_dct = dft_parser.parse_directory(dft_tuple)

labels, scores = rank_ff_scores(
    md_parsed_dct,
    dft_parsed_dct,
    energy_weight,
    force_weight,
)

out = {
    "labels": labels,
    "scores": scores,
}

print(json.dumps(out))
"""

valid_out = subprocess.check_output(
    [
        "/global/homes/r/rym/.conda/envs/unified/bin/python",
        "-c",
        validation_code,
        md_Validation_MACE_directory,
        path_to_aimd_sp_poscars,
        '(9, 14)', 
        '(7, 8)',
        '0.1', 
        '5.0'
    ],
    text=True
)

validation_data = json.loads(valid_out)

validation_labels = validation_data["labels"]
validation_scores = validation_data["scores"]

In [53]:
validation_labels, validation_scores

(['medium-omat-0/freeze_6/energy_1/5014/5014',
  'medium-omat-0/freeze_6/energy_1/5071/5071',
  'medium-omat-0/freeze_6/energy_3/314/314',
  'medium-omat-0/freeze_6/energy_2/3150/3150',
  'medium-omat-0/freeze_6/energy_2/5318/5318',
  'medium-omat-0/freeze_6/energy_3/6801/6801',
  'medium-omat-0/freeze_7/energy_3/6411/6411',
  'medium-omat-0/freeze_7/energy_3/9680/9680',
  'medium-omat-0/freeze_7/energy_2/6060/6060',
  'medium-omat-0/freeze_7/energy_2/3471/3471',
  'medium-omat-0/freeze_7/energy_1/4600/4600',
  'medium-omat-0/freeze_7/energy_1/4946/4946'],
 [0.33053787353811104,
  0.3657722811160213,
  0.3933208463369913,
  0.4220466567057933,
  0.4607792760536623,
  0.48506449263728363,
  0.7557975861353953,
  0.7774998629282369,
  0.7955082328339333,
  0.8036229797184363,
  0.8226987482093048,
  0.8246843855389048])

Having now ranked our force fields, we can select the next force field for fitting, and restart the fitting cycle with our most recent data additions and best performer. 

In [54]:
base_model_string = md_Validation_MACE_directory.replace(f'/{model_name}', '')
best_FF_full_path = os.path.join(base_model_string, validation_labels[0], model_string)
print(f'Path to best force field: {best_FF_full_path}, Exists: {os.path.exists(best_FF_full_path)}')

Path to best force field: /pscratch/sd/r/rym/Pd-Se_MACE/NB_example/MD/Validation/medium-omat-0/freeze_6/energy_1/5014/5014/model.model, Exists: True
